# LGBIO2060 Exercice session 1
__Authors:__ Simon Vandergooten and Clemence Vandamme

__Content inspired from__: Neuromatch Academy github.com/NeuromatchAcademy

Statistical reminder :
In this first exercise session we will refresh your memory about the basis of statistics needed for the rest of the sessions.
After this session you should be able to ...

## Imports and helper functions
**Please execute the cell(s) below to initialize the notebook environment.**

In [1]:
#Import the libraries 
import numpy as np #for the math stuff
import matplotlib.pyplot as plt #for the plot handling
from scipy.stats import norm

In [2]:
# @title Figure Settings
import ipywidgets as widgets  # interactive display
%config InlineBackend.figure_format = 'retina'
# use NMA plot style
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")
my_layout = widgets.Layout()

In [3]:
# @title Plotting Functions

def plot_random_sample(x, y, figtitle = None):
  """ Plot the random sample between 0 and 1 for both the x and y axes.

  Args:
      x (ndarray): array of x coordinate values across the random sample
      y (ndarray): array of y coordinate values across the random sample
      figtitle (str): title of histogram plot (default is no title)

    Returns:
      Nothing.
  """
  fig, ax = plt.subplots()
  ax.set_xlabel('x')
  ax.set_ylabel('y')
  plt.xlim([-0.25, 1.25]) # set x and y axis range to be a bit less than 0 and greater than 1
  plt.ylim([-0.25, 1.25])
  plt.scatter(dataX, dataY)
  if figtitle is not None:
    fig.suptitle(figtitle, size=16)
  plt.show()

def plot_random_walk(x, y, figtitle = None):
  """ Plots the random walk within the range 0 to 1 for both the x and y axes.

    Args:
      x (ndarray): array of steps in x direction
      y (ndarray): array of steps in y direction
      figtitle (str): title of histogram plot (default is no title)

    Returns:
      Nothing.
  """
  fig, ax = plt.subplots()
  plt.plot(x,y,'b-o', alpha = 0.5)
  plt.xlim(-0.1,1.1)
  plt.ylim(-0.1,1.1)
  ax.set_xlabel('x location')
  ax.set_ylabel('y location')
  plt.plot(x[0], y[0], 'go')
  plt.plot(x[-1], y[-1], 'ro')

  if figtitle is not None:
    fig.suptitle(figtitle, size=16)
  plt.show()

def plot_hist(data, xlabel, figtitle = None, num_bins = None):
  """ Plot the given data as a histogram.

    Args:
      data (ndarray): array with data to plot as histogram
      xlabel (str): label of x-axis
      figtitle (str): title of histogram plot (default is no title)
      num_bins (int): number of bins for histogram (default is 10)

    Returns:
      count (ndarray): number of samples in each histogram bin
      bins (ndarray): center of each histogram bin
  """
  fig, ax = plt.subplots()
  ax.set_xlabel(xlabel)
  ax.set_ylabel('Count')
  if num_bins is not None:
    count, bins, _ = plt.hist(data, bins = num_bins)
  else:
    count, bins, _ = plt.hist(data, bins = np.arange(np.min(data)-.5, np.max(data)+.6)) # 10 bins default
  if figtitle is not None:
    fig.suptitle(figtitle, size=16)
  plt.show()
  return count, bins

def my_plot_single(x, px):
  """
  Plots normalized Gaussian distribution

    Args:
        x (numpy array of floats):     points at which the likelihood has been evaluated
        px (numpy array of floats):    normalized probabilities for prior evaluated at each `x`

    Returns:
        Nothing.
  """
  if px is None:
      px = np.zeros_like(x)

  fig, ax = plt.subplots()
  ax.plot(x, px, '-', color='C2', LineWidth=2, label='Prior')
  ax.legend()
  ax.set_ylabel('Probability')
  ax.set_xlabel('Orientation (Degrees)')

def plot_gaussian_samples_true(samples, xspace, mu, sigma, xlabel, ylabel):
  """ Plot a histogram of the data samples on the same plot as the gaussian
  distribution specified by the give mu and sigma values.

    Args:
      samples (ndarray): data samples for gaussian distribution
      xspace (ndarray): x values to sample from normal distribution
      mu (scalar): mean parameter of normal distribution
      sigma (scalar): variance parameter of normal distribution
      xlabel (str): the label of the x-axis of the histogram
      ylabel (str): the label of the y-axis of the histogram

    Returns:
      Nothing.
  """
  fig, ax = plt.subplots()
  ax.set_xlabel(xlabel)
  ax.set_ylabel(ylabel)
  # num_samples = samples.shape[0]

  count, bins, _ = plt.hist(samples, density=True)
  plt.plot(xspace, norm.pdf(xspace, mu, sigma),'r-')
  plt.show()
    
def plot_likelihoods(likelihoods, mean_vals, variance_vals):
  """ Plot the likelihood values on a heatmap plot where the x and y axes match
  the mean and variance parameter values the likelihoods were computed for.

    Args:
      likelihoods (ndarray): array of computed likelihood values
      mean_vals (ndarray): array of mean parameter values for which the
                            likelihood was computed
      variance_vals (ndarray): array of variance parameter values for which the
                            likelihood was computed

    Returns:
      Nothing.
  """
  fig, ax = plt.subplots()
  im = ax.imshow(likelihoods)

  cbar = ax.figure.colorbar(im, ax=ax)
  cbar.ax.set_ylabel('log likelihood', rotation=-90, va="bottom")

  ax.set_xticks(np.arange(len(mean_vals)))
  ax.set_yticks(np.arange(len(variance_vals)))
  ax.set_xticklabels(mean_vals)
  ax.set_yticklabels(variance_vals)
  ax.set_xlabel('Mean')
  ax.set_ylabel('Variance')

def posterior_plot(x, likelihood=None, prior=None, posterior_pointwise=None, ax=None):
  """
  Plots normalized Gaussian distributions and posterior.

    Args:
        x (numpy array of floats):         points at which the likelihood has been evaluated
        auditory (numpy array of floats):  normalized probabilities for auditory likelihood evaluated at each `x`
        visual (numpy array of floats):    normalized probabilities for visual likelihood evaluated at each `x`
        posterior (numpy array of floats): normalized probabilities for the posterior evaluated at each `x`
        ax: Axis in which to plot. If None, create new axis.

    Returns:
        Nothing.
  """
  if likelihood is None:
      likelihood = np.zeros_like(x)

  if prior is None:
      prior = np.zeros_like(x)

  if posterior_pointwise is None:
      posterior_pointwise = np.zeros_like(x)

  if ax is None:
    fig, ax = plt.subplots()

  ax.plot(x, likelihood, '-C1', LineWidth=2, label='Auditory')
  ax.plot(x, prior, '-C0', LineWidth=2, label='Visual')
  ax.plot(x, posterior_pointwise, '-C2', LineWidth=2, label='Posterior')
  ax.legend()
  ax.set_ylabel('Probability')
  ax.set_xlabel('Orientation (Degrees)')
  plt.show()

  return ax

def plot_classical_vs_bayesian_normal(num_points, mu_classic, var_classic,
                                      mu_bayes, var_bayes):
  """ Helper function to plot optimal normal distribution parameters for varying
  observed sample sizes using both classic and Bayesian inference methods.

    Args:
      num_points (int): max observed sample size to perform inference with
      mu_classic (ndarray): estimated mean parameter for each observed sample size
                                using classic inference method
      var_classic (ndarray): estimated variance parameter for each observed sample size
                                using classic inference method
      mu_bayes (ndarray): estimated mean parameter for each observed sample size
                                using Bayesian inference method
      var_bayes (ndarray): estimated variance parameter for each observed sample size
                                using Bayesian inference method

    Returns:
      Nothing.
  """
  xspace = np.linspace(0, num_points, num_points)
  fig, ax = plt.subplots()
  ax.set_xlabel('n data points')
  ax.set_ylabel('mu')
  plt.plot(xspace, mu_classic,'r-', label = "Classical")
  plt.plot(xspace, mu_bayes,'b-', label = "Bayes")
  plt.legend()
  plt.show()

  fig, ax = plt.subplots()
  ax.set_xlabel('n data points')
  ax.set_ylabel('sigma^2')
  plt.plot(xspace, var_classic,'r-', label = "Classical")
  plt.plot(xspace, var_bayes,'b-', label = "Bayes")
  plt.legend()
  plt.show()

## Section 1: Probability distribution

This section will be devoted to the exploration of some probability distribution.

The goal is to give you the intuition about what is a probability distribution and why it can be usefull for modeling.

### Coding Exercise 1.1: Create randomness

Numpy has many functions and capabilities related to randomness.  We can draw random numbers from various probability distributions. For example, to draw 5 uniform numbers between 0 and 100, you would use `np.random.uniform(0, 100, size = (5,))`. 

 We will use `np.random.seed` to set a specific seed for the random number generator. For example, `np.random.seed(0)` sets the seed as 0. By including this, we are actually making the random numbers reproducible, which may seem odd at first. Basically if we do the below code without that 0, we would get different random numbers every time we run it. By setting the seed to 0, we ensure we will get the same random numbers. There are lots of reasons we may want randomness to be reproducible.

```
np.random.seed(0)
random_nums = np.random.uniform(0, 100, size = (5,))
```

Below, you will complete a function `generate_random_sample` that randomly generates `num_points` $x$ and $y$ coordinate values, all within the range 0 to 1. You will then generate 10 points and visualize.

In [ ]:
def generate_random_sample(num_points):
    """ Generate a random sample containing a desired number of points (num_points)
    in the range [0, 1] using a random number generator object.

     Args:
       num_points (int): number of points desired in random sample

     Returns:
       dataX, dataY (ndarray, ndarray): arrays of size (num_points,) containing x
       and y coordinates of sampled points

     """
    ###################################################################
    ## TODO for students: Draw the uniform numbers
    ###################################################################
    dataX = ...
    dataY = ...
    
    return dataX, dataY
    ###################################################################

# Set a seed
np.random.seed(0)

# Set number of points to draw
num_points = 10

# Draw random points
dataX, dataY = generate_random_sample(num_points)
    
# Visualize
with plt.xkcd():
    plot_random_sample(dataX, dataY, "Random sample of 10 points")

### Coding exercice 1.2: Gaussian Distribution

The most widely used continuous distribution is probably the Gaussian (also known as Normal) distribution. It is extremely common across all kinds of statistical analyses. Because of the central limit theorem, many quantities are Gaussian distributed. Gaussians also have some nice mathematical properties that permit simple closed-form solutions to several important problems. 

As a working example, imagine that a human participant is asked to point in the direction where they perceived a sound coming from. As an approximation, we can assume that the variability in the direction/orientation they point towards is Gaussian distributed.

In this exercise, you will implement a Gaussian by filling in the missing portions of code for the function `my_gaussian` below. Gaussians have two parameters. The **mean** $\mu$, which sets the location of its center, and its "scale" or spread is controlled by its **standard deviation** $\sigma$, or **variance** $\sigma^2$ (i.e. the square of standard deviation). **Be careful not to use one when the other is required.**

The equation for a Gaussian probability density function is:
$$
f(x;\mu,\sigma^2)=\mathcal{N}(\mu,\sigma^2) = \frac{1}{\sqrt{2\pi\sigma^2}}\exp\left(\frac{-(x-\mu)^2}{2\sigma^2}\right)
$$
In Python $\pi$ and $e$ can be written as `np.pi` and `np.exp` respectively.

As a probability distribution this has an integral of one when integrated from $-\infty$ to $\infty$, however in the following your numerical Gaussian will only be computed over a finite number of points. You therefore need to explicitly normalize it to sum to one yourself based on the `step_size` used. 

You can test your function with $\mu= 1$ and $\sigma = 1.5$

In [ ]:
def my_gaussian(x_points, mu, sigma):
    """ Returns normalized Gaussian estimated at points `x_points`, with
    parameters: mean `mu` and standard deviation `sigma`

    Args:
      x_points (ndarray of floats): points at which the gaussian is evaluated
      mu (scalar): mean of the Gaussian
      sigma (scalar): standard deviation of the gaussian

    Returns:
      (numpy array of floats) : normalized Gaussian evaluated at `x`
    """
    ###################################################################
    ## TODO for students: Implement the gaussian equation
    ###################################################################
    px = ...
    
    #Do not forget to normalize
    
    return px
#Choose some parameters
mu = ...
sigma = ...
   
###################################################################
#Create data  
step_size = 0.1
x_points = np.arange(-5, 5, step_size)

#Generate the Gaussian
Gaussian = my_gaussian(x_points, mu, sigma)

#Visualise
my_plot_single(x, Gaussian)

Questions:
* What changes do you observe by varying the mean ?
* What changes do you observe by varying the variance ?

## Section 2: Statistical inference
The goal of this section is to explain how it is possible to do inference by inverting the generative process.

By completing the exercises in this tutorial, you should:
* understand what the likelihood function is, and have some intuition of why it is important
* know how to summarise the Gaussian distribution using mean and variance 
* know how to maximise a likelihood function
* be able to do simple inference in both classical and Bayesian ways

### Section 2.1: Basic rules of probability

Definitions: 
* $P(A) \in [ \,0,1 ]\,$ is the probability that event A is true.
* $P(\neg\, A) = 1 - P(A)$ is the **complementary probability** of the event A.
* $P(A \cap B) = P(A | B)  P(B) = P(B | A)P(A) = P(A,B)$ is the **joint probability** of events A and B. Meaning both events are true.
* $P(A | B) = P(A \cap B)/P(B)$ is the **conditionnal probability** of the event A being true given the event B is true.
* $P(A) = \sum_{B} P(A,B) = \sum_B P(A|B)P(B)$ is the **marginal probability** of the event A.

### Math exercice 2.1: Example
In order to check if those formula are clear to you, let's take a small example to illustrate them.

Imagine that for the past 10 years, hundreds of students' opinions on the LGBIO2060 course have been gathered aswell as if they have pass the exam. It turns out that on average 75% of the students actually enjoyed the course and 60% succeed at the exam (Those numbers are fictional).

We will use the following notations:

$P(E)=0.75$ is the probability that students enjoyed the course.

$P(S)=0.6$ is the probability that students succeed at the exam.

#### A) Product
Assuming that enjoying the course and passing the exam are two indepedent events, what is the probability that a randomly chosen student enjoyed the course and passed the exam ? i.e $P(E \cap S)$

In [8]:
#Enter your answer here
p_joint = 0.45

In [9]:
# @title Test your answer
if p_joint == 0.45:
    print('Your answer is correct, congratulations !!!')
else:
    print('Ho :( Your answer is not the expected one')

Your answer is correct, congratulations !!!
